<a href="https://colab.research.google.com/github/deckboy/NLP/blob/main/Sentiment_Classification_using_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 1.6MB/s 
     |████████████████████████████████| 6.8MB 43.8MB/s 
     |████████████████████████████████| 983kB 33.4MB/s 
     |████████████████████████████████| 266kB 50.3MB/s 
     |████████████████████████████████| 1.9MB 36.6MB/s 
     |████████████████████████████████| 1.2MB 37.4MB/s 
     |████████████████████████████████| 471kB 51.7MB/s 
     |████████████████████████████████| 3.3MB 38.4MB/s 
     |████████████████████████████████| 901kB 41.2MB/s 
  Created wheel for ktrain: filename=ktrain-0.26.3-cp37-none-any.whl size=25282390 sha256=fd498f626e2ad9973c571708d3c903a4cce5aec8bea24b27e53267c70061360e
  Stored in directory: /root/.cache/pip/wheels/16/05/be/d6e659b3349016b1059e19fa028f165af4eeae2c196f329112
  Created wheel for langdetect: filename=langdetect-1.0.9-cp37-none-any.whl size=993242 sha256=cd9cd898a1c7b21a4341385a833d2b0bf06ff12c3cfbe847afe43772e2e9d952
  Stored in directory: /root/.cache/pip/wheels/7e/18/13/038c34057808931c7d

In [2]:
import tensorflow as tf

In [3]:
tf.__version__

'2.5.0'

In [4]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git

Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [5]:
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf

In [6]:
data_train = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx', dtype = str)

In [7]:
data_test = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx', dtype = str)

In [8]:
data_train.head()

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg


In [9]:
data_test.head()

,Reviews,Sentiment
0,Who would have thought that a movie about a ma...,pos
1,After realizing what is going on around us ......,pos
2,I grew up watching the original Disney Cindere...,neg
3,David Mamet wrote the screenplay and made his ...,pos
4,"Admittedly, I didn't have high expectations of...",neg


In [10]:
data_test.shape, data_train.shape

((25000, 2), (25000, 2))

In [11]:
data_train[data_train.Sentiment =='pos'].shape

(12500, 2)

In [12]:
data_train[data_train.Sentiment =='neg'].shape

(12500, 2)

In [13]:
data_test[data_train.Sentiment =='neg'].shape

(12500, 2)

In [14]:
(X_train, y_train), (X_test, y_test), prepro = text.texts_from_df(train_df = data_train, 
                                                                  text_column='Reviews', 
                                                                  label_columns='Sentiment', 
                                                                  val_df = data_test, 
                                                                  maxlen=500, 
                                                                  preprocess_mode='bert')

['neg', 'pos']
   neg  pos
0  1.0  0.0
1  1.0  0.0
2  1.0  0.0
3  1.0  0.0
4  1.0  0.0
['neg', 'pos']
   neg  pos
0  0.0  1.0
1  0.0  1.0
2  1.0  0.0
3  0.0  1.0
4  1.0  0.0
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [15]:
model = text.text_classifier(name = 'bert', train_data = (X_train, y_train), preproc = prepro)

Is Multi-Label? False
maxlen is 500
done.


In [16]:
learner = ktrain.get_learner(model = model, 
                             train_data = (X_train, y_train), 
                             val_data = (X_test, y_test), 
                             batch_size = 6)

In [17]:
#learner.lr_find()
#learner.lr_plot()

In [18]:
learner.fit_onecycle(lr = 2e-5, epochs = 1)



begin training using onecycle policy with max lr of 2e-05...
4167/4167 [==============================] - 2366s 563ms/step - loss: 0.2529 - accuracy: 0.8921 - val_loss: 0.1022 - val_accuracy: 0.9690


In [20]:
predictor = ktrain.get_predictor(learner.model, prepro)

In [22]:
data = ['this moview was horrible, the plot was really boring. acting was okay', 
        'the film really sucked. there is not plot and acting was bad',
        'what a beautiful movie. great plot. acting was good. will see it again']

In [23]:
predictor.predict(data)

['neg', 'neg', 'pos']

In [24]:
predictor.predict(data, return_proba=True)

array([[0.9957027 , 0.0042973 ],
       [0.9936114 , 0.00638865],
       [0.00267806, 0.9973219 ]], dtype=float32)

In [25]:
predictor.get_classes()

['neg', 'pos']

In [26]:
predictor.save('/content/drive/MyDrive/Algorithms/nlp/bert')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [27]:
predictor_load = ktrain.load_predictor('/content/drive/MyDrive/Algorithms/nlp/bert')

In [28]:
predictor_load.get_classes()

['neg', 'pos']

In [29]:
predictor_load.predict(data)

['neg', 'neg', 'pos']